Typical Process
- Set up API keys for search tool, open_api and Langsmith
- Set up search tool using tavily search and attach return_direct feature
- Set up model, bind tools to model, and bind specific response format to tool
- Define the agent state
- Define nodes, that is, functions to be invoked on each node
- Define graph workflow
- Use it

## Set up

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API Key:")

OpenAI API Key: ········
Tavily API Key: ········


In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key:")

LangSmith API Key: ········


## Set up tool

In [48]:
from pydantic import BaseModel, Field

class SearchTool(BaseModel):
    """Look things up online, optionally return directly to user"""
    query: str = Field(description="The query passed to the search tool")
    return_direct: bool = Field(
        description="Whether or not the result of the search tool should be returned directly to the user",
        default=False
    )

In [49]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(max_results=1, args_schema=SearchTool)
tools = [search_tool]

In [50]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

## Set up model
Bind a structured response

In [51]:
from pydantic import BaseModel, Field

class Response(BaseModel):
    """The information needed from search tool"""
    
    temperature: float = Field(description="The temperature of weather")
    pressure: float = Field(description="The pressure of weather")
    other_notes: str = Field(description="Notes on general condition of the weather")

In [52]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, streaming=True)

In [53]:
from  langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tools]
functions.append(convert_to_openai_function(Response))
model_with_tools = model.bind_functions(functions)

## Define the agent state

In [54]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

## Define the nodes

In [55]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

**MODIFICATION**

We will change the `should_continue` function to check what function was called. If the function `Response` was called - that is the function that is NOT a tool, but rather the formatted response, so we should NOT continue in that case.

In [56]:
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]

    if "function_call" in last_message.additional_kwargs:
        print("Name of function call: ", last_message.additional_kwargs["function_call"]["name"])
    
    # If no function call, end the workflow
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # If last call is a Response, end the workflow
    elif last_message.additional_kwargs["function_call"]["name"] == "Response":
        return "end"
    else:
        # Hence, there is a function call
        arguments = json.loads(
            last_message.additional_kwargs["function_call"]["arguments"]
        )
        if arguments.get("return_direct"):
            return "final"
        else:
            return "continue"
    

In [57]:
# Define the function that calls the model
def call_model(state):
    messages = state["messages"]
    response = model_with_tools.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Define the function to execute tools
def call_tool(state):
    messages = state["messages"]
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(
            last_message.additional_kwargs["function_call"]["arguments"]
        ),
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

## Define the graph
We can now put it all together and define the graph!

In [58]:
from langgraph.graph import StateGraph, END

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)
workflow.add_node("final", call_tool)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Final call
        "final": "final",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")
workflow.add_edge("final", END)

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [59]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the weather in San Fransciso, United States")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in San Francisco, United States"}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call'})]}

---

Name of function call:  tavily_search_results_json
Output from node 'action':
---
{'messages': [FunctionMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1712827292, \'localtime\': \'2024-04-11 2:21\'}, \'current\': {\'last_updated_epoch\': 1712826900, \'last_updated\': \'2024-04-11 02:15\', \'temp_c\': 12.2, \'temp_f\': 54.0, \'is_day\': 0, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/116.png\', \'code\': 1003}, \'wind_mph\': 9.4, \'wind